In [ ]:
!pip install -qU json-repair==0.29.1
!pip install -q datasets

In [3]:
import pandas as pd
import json
from os.path import join
import os

In [4]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime

import json_repair

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [5]:
import json_repair

def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None

In [42]:
df = pd.read_csv('/content/drive/MyDrive/LLMs/datasets/combined_jobs.csv')

In [9]:
data_dir = os.path.join(os.getcwd(), 'drive/MyDrive/LLMs/datasets')

In [10]:
data_dir

'/content/drive/MyDrive/LLMs/datasets'

In [11]:
df.head()

,Position,Company_Name,Location,Post_Month,Post_Year,Details
0,IT Manager,10 Percent Recruiting Ltd.,"Vancouver, British Columbia, Canada",June,2024,Position Title: IT Manager\n\nLocation: Vancou...
1,"Manager, IT Support",Procom,"Toronto, Ontario, Canada",June,2024,"On behalf of our public sector client, PROCOM ..."
2,Director of Information Technology,Southampton Financial Inc,"Toronto, Ontario, Canada",June,2024,Southampton Financials’ Mission: Bring Clarity...
3,"Manager, Information Technology Services",Town of Tillsonburg,"Tillsonburg, Ontario, Canada",June,2024,The Town of Tillsonburg is looking for a Manag...
4,Systems Manager,Accor,"Winnipeg, Manitoba, Canada",June,2024,"Company Description\n\n""Why work for Accor?""\n..."


In [12]:
df.columns

Index(['Position', 'Company_Name', 'Location', 'Post_Month', 'Post_Year',
       'Details'],
      dtype='object')

In [43]:
example = df['Details'][0]

In [14]:
df = df.head(300)

In [15]:
df.shape

(300, 6)

In [16]:
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 11.0 MB/s eta 0:00:00


## Structured Output template

In [17]:
{
  "role_summary": "A concise, non-technical summary of the job role. It should describe the primary responsibilities of the role in simple language, avoiding jargon. Focus on what the role does rather than listing requirements.",
  "key_terms": [
    {
      "term": "Technical Term or jargon from the job description",
      "explanation": "A simple explanation of the term in the context of the role. This helps recruiters understand technical jargon without needing domain expertise."
    }
  ],
  "skill_priorities": {
    "must_have": ["List of essential skills required for the role. These are non-negotiable and should be explicitly mentioned in the job description."],
    "nice_to_have": ["List of preferred skills that are beneficial but not mandatory. These are often marked as 'preferred,' 'a plus,' or 'optional' in the job description."]
  },
  "proposed_screening_questions_with_answers": [
    {
      "question": "A role-specific question to assess candidate expertise. The question should focus on technical skills, problem-solving, or past experiences relevant to the role.",
      "example_answer": "An example of a strong candidate response to the question. The answer should demonstrate technical depth, problem-solving, and relevance to the role."
    }
  ],
  "red_flags": [
    "Indicators of potential mismatches for the role, offering actionable insights for recruiters based on the job offer. Examples include 'Avoid candidates without cloud experience'."
  ],
  "confidence_score": "A numerical score between 0 and 100 indicating the model's confidence in the accuracy of its analysis."
}


{'role_summary': 'A concise, non-technical summary of the job role. It should describe the primary responsibilities of the role in simple language, avoiding jargon. Focus on what the role does rather than listing requirements.',
 'key_terms': [{'term': 'Technical Term or jargon from the job description',
   'explanation': 'A simple explanation of the term in the context of the role. This helps recruiters understand technical jargon without needing domain expertise.'}],
 'skill_priorities': {'must_have': ['List of essential skills required for the role. These are non-negotiable and should be explicitly mentioned in the job description.'],
  'nice_to_have': ["List of preferred skills that are beneficial but not mandatory. These are often marked as 'preferred,' 'a plus,' or 'optional' in the job description."]},
 'proposed_screening_questions_with_answers': [{'question': 'A role-specific question to assess candidate expertise. The question should focus on technical skills, problem-solving

In [18]:

from pydantic import BaseModel, Field
from typing import List, Optional, Literal

class key_terms(BaseModel):
  term : str = Field(..., description="Technical Term or jargon from the job description")
  explanation : str = Field(..., description="A simple explanation of the term in the context of the role. This helps recruiters understand technical jargon without needing domain expertise.")

class skill_priorities(BaseModel):
  must_have : List[str] = Field(..., description="List of essential skills required for the role. These are non-negotiable and should be explicitly mentioned in the job description.")
  nice_to_have : List[str] = Field(..., description="List of preferred skills that are beneficial but not mandatory. These are often marked as 'preferred,' 'a plus,' or 'optional' in the job description.")

class proposed_screening_questions_with_answers(BaseModel):
  question : str = Field(..., description="A role-specific question to assess candidate expertise. The question should focus on technical skills, problem-solving, or past experiences relevant to the role.")
  example_answer : str = Field(..., description="An example of a strong candidate response to the question. The answer should demonstrate technical depth, problem-solving, and relevance to the role.")


class JobAnalysis(BaseModel):
  role_summary : str
  key_terms : List[key_terms]
  skill_priorities : skill_priorities
  proposed_screening_questions_with_answers : List[proposed_screening_questions_with_answers]
  red_flags : List[str]
  confidence_score : float



## design of the extraction prompts

In [19]:
extraction_messages = [
    {
        "role" : "system",
        "content":"\n".join([
            "You are an AI assistant that converts job descriptions into structured JSON data to assist recruiters.",
            "Your task is to extract key details from a given job description and format them into a JSON object with the following structure."
            "The Output Structure must follow the provided pydantic details ",
        ])
    },
    {
        "role" : "user",
        "content" : "\n".join([
            "## Job Description:",
            example,
            ""

            "## Output Structure:",
            json.dumps(
                JobAnalysis.model_json_schema()
            ),
            ""


            "## Extraction Details: ",
            ""


        ])
    }
]


## Preparing the Dataset : Knowledge Distillation

In [20]:
from groq import Groq
from google.colab import userdata
# Initialize client
client = Groq(api_key=userdata.get('GROQ_API_KEY'))

# Chat completion using LLaMA 3
response = client.chat.completions.create(
    model="gemma2-9b-it",  # or other models like gemma-7b-it
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What's the capital of Tunisia?"}
    ]
)

# Print the reply
print(response.choices[0].message.content)


The capital of Tunisia is **Tunis**. 😄  



In [7]:
models = {
    "gemma_mdoel" : "gemma2-9b-it",
    "llama_model" : "llama2-7b-chat",
    "mistral_model" : "mistral-7b-instruct"
}


In [6]:
def extract_json(messages, model):
    response = client.chat.completions.create(
        model=model,  # or other models like gemma-7b-it
        messages=messages
    )

    # Print the reply
    return response.choices[0].message.content

In [ ]:
result = extract_json(extraction_messages, models['gemma_mdoel'])

result

'```json\n{\n  "RoleSummary": "Dynamic and communicative IT Manager to join an engineering team. Requires hands-on technical expertise, vendor management mastery, exceptional stakeholder engagement, and familiarity with engineering environments.",\n  "key_terms": [\n    {\n      "term": "MSP",\n      "explanation": "Managed Service Provider, outsourcing IT services like network administration or helpdesk support."\n    },\n    {\n      "term": "MSSP",\n      "explanation": "Managed Security Service Provider, specializing in cybersecurity services like threat detection and response."\n    },\n    {\n      "term": "AutoCad",\n      "explanation": "Popular software for drafting and designing 2D and 3D drawings commonly used in engineering."\n    },\n    {\n      "term": "Rhino",\n      "explanation": "3D modeling software widely used in engineering, product design, and manufacturing."\n    }\n  ],\n  "skill_priorities": {\n    "must_have": [\n      "Technical Expertise",\n      "Vendor Ma

In [ ]:
parsed_jons_result = parse_json(result)

parsed_jons_result

{'RoleSummary': 'Dynamic and communicative IT Manager to join an engineering team. Requires hands-on technical expertise, vendor management mastery, exceptional stakeholder engagement, and familiarity with engineering environments.',
 'key_terms': [{'term': 'MSP',
   'explanation': 'Managed Service Provider, outsourcing IT services like network administration or helpdesk support.'},
  {'term': 'MSSP',
   'explanation': 'Managed Security Service Provider, specializing in cybersecurity services like threat detection and response.'},
  {'term': 'AutoCad',
   'explanation': 'Popular software for drafting and designing 2D and 3D drawings commonly used in engineering.'},
  {'term': 'Rhino',
   'explanation': '3D modeling software widely used in engineering, product design, and manufacturing.'}],
 'skill_priorities': {'must_have': ['Technical Expertise',
   'Vendor Management',
   'Stakeholder Engagement',
   'IT Project Management',
   'Cybersecurity',
   'Data Management'],
  'nice_to_have'

In [23]:
raw_data = []

for item in df['Details']:
  raw_data.append(item)


In [24]:
import json
from os.path import join

output_path = join(data_dir, 'jobs_descriptions.jsonl')

with open(output_path, 'w') as f:
    for item in raw_data:
        if item and isinstance(item, str) and item.strip():
            json_line = json.dumps({"description": item.strip()})
            f.write(json_line + "\n")

In [25]:
output_path

'/content/drive/MyDrive/LLMs/datasets/jobs_descriptions.jsonl'

In [26]:
len(raw_data)

300

In [27]:
import random

new_raw_data = []
for line in open(output_path):
    if line.strip() == "":
        continue

    new_raw_data.append(
        json.loads(line.strip())
    )

random.Random(101).shuffle(new_raw_data)

print(f"Raw data: {len(new_raw_data)}")

Raw data: 300


In [ ]:
model = models['gemma_mdoel']

save_to = join(data_dir, "job_descriptions", "job_descriptions_fine_tuning_data.jsonl")


for story in tqdm(new_raw_data):

    sample_details_extraction_messages = [
        {
            "role": "system",
            "content": "\n".join([
                "You are an AI assistant that converts job descriptions into structured JSON data to assist recruiters.",
                "Your task is to extract key details from a given job description and format them into a JSON object with the following structure."
                "The Output Structure must follow the provided pydantic details ",
            ])
        },
        {
            "role": "user",
            "content": "\n".join([
                "## Story:",
                story['description'].strip(),
                "",

                "## Output Structure:",
                json.dumps(
                    JobAnalysis.model_json_schema(), ensure_ascii=False
                ),
                "",

                "## Extraction Details: ",
                "```json"
            ])
        }
    ]

    response = extract_json(sample_details_extraction_messages, model)

    llm_response = response
    llm_resp_dict = parse_json(llm_response)

    if not llm_resp_dict:
        continue

    with open(save_to, "a", encoding="utf8") as dest:
        dest.write(json.dumps({
            "description": story['description'].strip(),
            "task": "Extrat the job description details into a JSON.",
            "output_scheme": json.dumps(JobAnalysis.model_json_schema()),
            "response": llm_resp_dict,
                                        }, ensure_ascii=False, default=str)  + "\n" )



## Examples of the distilled data created for the training using gemma2-9b-it

In [44]:
sft_data_path = os.path.join(data_dir, "job_descriptions", "job_descriptions_fine_tuning_data.jsonl")
with open(sft_data_path, "r") as f:
    lines = f.readlines()

In [45]:
data = [json.loads(line) for line in lines]

In [47]:
data[0]['description']

"Benefits:\n\nBonus based on performance\nCompetitive salary\nHome office stipend\nPaid time off\nTraining & development\n\n\nWe are seeking a DevOps Engineer to join our growing team and take ownership of our cloud infrastructure, CI/CD pipelines, system reliability, observability, and scalability. The ideal candidate is passionate about automation, cloud technologies, and security best practices, ensuring seamless deployment and high availability of our applications.\n\nResponsibilities\n\nDesign, implement, and manage CI/CD pipelines to streamline software development and deployment.\nMaintain and optimize cloud infrastructure (AWS, Azure) to ensure scalability, security, and cost-effectiveness.\nAutomate infrastructure provisioning, monitoring, and management using Infrastructure as Code (IaC) tools (Terraform, Ansible, etc.).\nMonitor system performance, troubleshoot issues, and ensure high availability and reliability.\nCollaborate with software engineers to enhance deployment st

In [31]:
data[1]

{'description': "Reports to: Vice President of Operations \n About Aroris: \n Aroris Health is a leading healthcare technology company. Our proprietary platform, Aroris360, empowers healthcare providers with data-driven insights to optimize payer negotiations, enhance operational efficiency, and improve healthcare outcomes. We are driven by innovation, committed to excellence, and focused on delivering measurable impact to our clients.  \n Aroris's mission is to preserve the spirit of medicine. One way of doing this is to provide independent medical practices access to the same payer contracting resources as the largest health systems. We operate as an extension of practices, leveraging decades of industry experience, cutting-edge data analytics capabilities, and dedicated legal and negotiation teams to help practices capture more revenue without expending additional resources. \n At Aroris, we look for individuals who embody our core values of being driven, joyful, relentless, and tea

## Fine Tuning using Unsloth

In [11]:
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"

Prepare data for fine tune

In [14]:
from datasets import Dataset

sft_data_path = os.path.join(data_dir, "job_descriptions", "job_descriptions_fine_tuning_data.jsonl")
with open(sft_data_path, "r") as f:
    lines = f.readlines()

system_message = "\n".join([
    "You are an AI assistant that converts job descriptions into structured JSON data to assist recruiters.",
    "Follow the provided `Task` by the user and the `Output Scheme` to generate the `Output JSON`.",
    "The Output Structure must follow the provided pydantic details."
])

# Prepare and shuffle data
llm_finetuning_data = []

for line in lines:
    if line.strip() == "":
        continue

    rec = json.loads(line.strip())

    instruction = "\n".join([
        system_message,
        "",
        "# description:",
        rec["description"],
        "",
        "# Task:",
        rec["task"],
        "",
        "# Output Scheme:",
        rec["output_scheme"],
        "",
        "# Output JSON:"

    ])

    output = json.dumps(rec["response"], ensure_ascii=False, indent=2)

    llm_finetuning_data.append({
        "instruction": instruction,
        "input": "",
        "output": output
    })

random.Random(101).shuffle(llm_finetuning_data)

# Convert to Hugging Face dataset
dataset = Dataset.from_list(llm_finetuning_data)

# Prepare for Unsloth fine-tuning with Alpaca-style prompt


In [15]:
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 112
})

In [16]:
print(dataset[0])

{'instruction': 'You are an AI assistant that converts job descriptions into structured JSON data to assist recruiters.\nFollow the provided `Task` by the user and the `Output Scheme` to generate the `Output JSON`.\nThe Output Structure must follow the provided pydantic details.\n\n# description:\nCompany Description\n\nAt Sleep Country Canada/Dormez-vous? (SCC/DV), we are inspired every day through our purpose to transform lives by awakening Canadians to the power of sleep and our vision to champion sleep as the key to healthier and happier lives, helping everyone achieve better tomorrows through better tonight’s.\n\nGuided by our values – We CARE About People; We WIN Together; We DREAM Big and We DELIVER with Excellence – we are building on our 30-year foundation of taking care of each other and our customers’ sleep needs, with passion and commitment to be the best that we can be. We invest in our sleep ecosystem, innovative products, world-class customer experience, our communities 

In [17]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [18]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [19]:
!pip install -qU bitsandbytes

In [20]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

<ipython-input-20-e8ea81163a91>:1: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/106k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.54G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [32]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [22]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)
    return { "text": texts }

# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

In [23]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 112
})

In [33]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 2,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 14,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/112 [00:00<?, ? examples/s]

In [34]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
9.652 GB of memory reserved.


In [35]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 112 | Num Epochs = 1 | Total steps = 14
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/7,000,000,000 (0.58% trained)


Step,Training Loss
10,1.088700


In [36]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

568.0398 seconds used for training.
9.47 minutes used for training.
Peak reserved memory = 9.652 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 65.477 %.
Peak reserved memory for training % of max memory = 0.0 %.


## Inference

In [48]:
instruction = "\n".join([
    "You are an AI assistant that converts job descriptions into structured JSON data to assist recruiters.",
    "Follow the provided `Task` by the user and the `Output Scheme` to generate the `Output JSON`.",
    "The Output Structure must follow the provided pydantic details.",
    "# description:",
        data[0]['description'],
        "",
        "# Task:",
        data[0]['task'],
        "",
        "# Output Scheme:",
        data[0]["output_scheme"],
        ""

])

In [50]:
instruction

'You are an AI assistant that converts job descriptions into structured JSON data to assist recruiters.\nFollow the provided `Task` by the user and the `Output Scheme` to generate the `Output JSON`.\nThe Output Structure must follow the provided pydantic details.\n# description:\nBenefits:\n\nBonus based on performance\nCompetitive salary\nHome office stipend\nPaid time off\nTraining & development\n\n\nWe are seeking a DevOps Engineer to join our growing team and take ownership of our cloud infrastructure, CI/CD pipelines, system reliability, observability, and scalability. The ideal candidate is passionate about automation, cloud technologies, and security best practices, ensuring seamless deployment and high availability of our applications.\n\nResponsibilities\n\nDesign, implement, and manage CI/CD pipelines to streamline software development and deployment.\nMaintain and optimize cloud infrastructure (AWS, Azure) to ensure scalability, security, and cost-effectiveness.\nAutomate in

In [57]:
import re

In [59]:
# alpaca_prompt = Copied from above

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

match = re.search(r"### Response:\s*(\{.*)", decoded, re.DOTALL)
if match:
    clean_output = match.group(1).strip()
else:
    clean_output = decoded.strip()

print(clean_output)

{
  "JobAnalysis": {
    "role_summary": "DevOps Engineer to join our growing team and take ownership of cloud infrastructure, CI/CD pipelines, system reliability, observability, and scalability.",
    "key_terms": [
      {
        "term": "CI/CD",
        "explanation": "Continuous Integration/Continuous Deployment, a set of practices for automating the build, test, and deployment of software."
      },
      {
        "term": "IaC",
        "explanation": "Infrastructure as Code, a method of managing and provisioning infrastructure using code."
      },
      {
        "term": "ELK stack",
        "explanation": "Elasticsearch, Logstash, and Kibana, a popular open-source platform for log management and analysis."
      },
      {
        "term": "Terraform",
        "explanation": "An infrastructure as code tool for provisioning, configuring, and managing infrastructure."
      },
      {
        "term": "Ansible",
        "explanation": "An open-source automation tool for configuri

In [73]:
job_desc_exp = open("/content/job_desc.txt", "r")
job_desc_exp_from_the_web = job_desc_exp.read()

In [74]:
job_desc_exp_from_the_web

'Reports to: Director of Marketing Analytics\nAbout Nexora:\nNexora is a fast-growing technology company revolutionizing the way businesses engage with customers through AI-powered marketing solutions. Our flagship platform, NexoraPulse, enables brands to personalize campaigns at scale, harnessing advanced analytics and automation. We are dedicated to fostering innovation, collaboration, and customer-centricity across all areas of our business.\n\nAt Nexora, we believe in building a workplace where curiosity is celebrated, impact is rewarded, and every team member is empowered to thrive. Our values—ownership, agility, inclusivity, and innovation—guide us in everything we do.\n\nJob Overview:\nWe are seeking a skilled and proactive Marketing Data Analyst to join our Analytics team. In this role, you will be responsible for collecting, interpreting, and visualizing marketing performance data to drive strategic decisions. You will work closely with marketing, product, and sales teams to p

In [75]:
instruction_001 = "\n".join([
    "You are an AI assistant that converts job descriptions into structured JSON data to assist recruiters.",
    "Follow the provided `Task` by the user and the `Output Scheme` to generate the `Output JSON`.",
    "The Output Structure must follow the provided pydantic details.",
    "# description:",
        job_desc_exp_from_the_web,
        "",
        "# Task:",
        data[0]['task'],
        "",
        "# Output Scheme:",
        data[0]["output_scheme"],
        ""

])

In [76]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction_001, # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1512, use_cache = True)
decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

match = re.search(r"### Response:\s*(\{.*)", decoded, re.DOTALL)
if match:
    clean_output = match.group(1).strip()
else:
    clean_output = decoded.strip()

print(clean_output)

{
  "JobAnalysis": {
    "role_summary": "The Marketing Data Analyst will be responsible for collecting, interpreting, and visualizing marketing performance data to drive strategic decisions. They will work closely with marketing, product, and sales teams to provide actionable insights and improve ROI on marketing campaigns.",
    "key_terms": [
      {
        "term": "CRM",
        "explanation": "Customer Relationship Management system used to manage customer interactions and data."
      },
      {
        "term": "BI",
        "explanation": "Business Intelligence tools used for data analysis and reporting."
      },
      {
        "term": "A/B Testing",
        "explanation": "A method of comparing two versions of a marketing campaign to determine which performs better."
      }
    ],
    "skill_priorities": {
      "must_have": [
        "SQL",
        "Excel",
        "Tableau",
        "Python/R"
      ],
      "nice_to_have": [
        "Google Analytics",
        "Power BI"

In [60]:
#get userdata
from google.colab import userdata
HF_TOKEN = userdata.get('HF_WRITE_TOKEN')


In [61]:
model.push_to_hub("mahdihammi/jobs_Qwen_lora_model", token = HF_TOKEN) # Online saving
tokenizer.push_to_hub("mahdihammi/jobs_Qwen_lora_model", token = HF_TOKEN)

README.md:   0%|          | 0.00/593 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/162M [00:00<?, ?B/s]

Saved model to https://huggingface.co/mahdihammi/jobs_Qwen_lora_model


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

## Load pushed model


In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "mahdihammi/jobs_Qwen_lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1028)